In [11]:
import sys
sys.path.insert(0, '..')
import os
import glob
from filament import filament
import datetime
import netCDF4
import cmocean
import logging
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from importlib import reload
reload(filament)

<module 'filament.filament' from '../filament/filament.py'>

In [3]:
logger = logging.getLogger("eddyPlot")
logger.setLevel(logging.INFO)
logging.info("Starting")
plotvis = 1

In [5]:
datadir = "/data/SST/Balearic/1/"
figdir = os.path.join(datadir, "figures")
visibledir = "/data/Visible/"
datafilelist = sorted(glob.glob(os.path.join(datadir, "*SST*.nc")))
datafilelist2 = sorted(glob.glob(os.path.join(datadir, "*SNPP*OC*.nc")))
nfiles = len(datafilelist)
logger.info("Processing {} files".format(nfiles))

if not(os.path.exists(figdir)):
    os.makedirs(figdir)

INFO:eddyPlot:Processing 1 files


In [6]:
coordinates = (2., 10., 37., 42.)
m = Basemap(projection='merc', llcrnrlon=coordinates[0], llcrnrlat=coordinates[2],
            urcrnrlon=coordinates[1], urcrnrlat=coordinates[3],
            epsg=4326, resolution='h')

In [54]:
for datafile in datafilelist:
    print(datafile)
    with netCDF4.Dataset(datafile) as nc:
        proclevel = nc.processing_level
        
    sst = filament.SST()
    
    # Select the reader according to processing level
    if proclevel == "L3 Mapped":
        logger.info("Level-3 data")
        sst.read_from_oceancolorL3(datafile, coordinates)
    elif proclevel == "L2":
        logger.info("Level-2 data")
        sst.read_from_oceancolorL2(datafile)
        sst.field = np.ma.masked_where(sst.qflag > 1, sst.field)
    
    # Plot without visible
    figname = os.path.join(figdir, sst.get_figname())
    sst.make_plot(m, figname=figname, vmin=20, vmax=25., shrink=0.7)

INFO:eddyPlot:Level-2 data


/data/SST/Balearic/1/T2019284214500.L2_LAC_SST.nc


## Chlorophyll

In [17]:
for datafile in datafilelist2:
    print(datafile)
    
    chloro = filament.Chloro()
    chloro.read_from_oceancolorL2(datafile)
    #lon, lat, chloro = read_chloro_from_oceancolorL2(datafile)

    fig = plt.figure(figsize=(8, 8))
    ax = plt.subplot(111)
    pcm = m.pcolormesh(chloro.lon, chloro.lat, chloro.field, 
                       latlon=True, vmin=0, vmax=0.6,
                       cmap=plt.cm.RdYlBu_r, zorder=6)
    plt.colorbar(pcm, extend="max", shrink=0.6)
    
    plt.show()